In [43]:
import cv2
import numpy as np
import os

In [44]:
# Specify the directory containing the UCF50 dataset
DATASET_DIR = "/kaggle/input/realistic-action-recognition-ucf50/UCF50"

In [45]:
!ls "/kaggle/input/realistic-action-recognition-ucf50/UCF50"

BaseballPitch  HorseRiding     PlayingPiano	   Skiing
Basketball     HulaHoop        PlayingTabla	   Skijet
BenchPress     JavelinThrow    PlayingViolin	   SoccerJuggling
Biking	       JugglingBalls   PoleVault	   Swing
Billiards      JumpingJack     PommelHorse	   TaiChi
BreastStroke   JumpRope        PullUps		   TennisSwing
CleanAndJerk   Kayaking        Punch		   ThrowDiscus
Diving	       Lunges	       PushUps		   TrampolineJumping
Drumming       MilitaryParade  RockClimbingIndoor  VolleyballSpiking
Fencing        Mixing	       RopeClimbing	   WalkingWithDog
GolfSwing      Nunchucks       Rowing		   YoYo
HighJump       PizzaTossing    SalsaSpin
HorseRace      PlayingGuitar   SkateBoarding


In [46]:

def frames_extraction(video_path, sequence_length=20, image_height=224, image_width=224):
    '''
    This function extracts the required frames from a video, resizes and normalizes them for model input.
    Args:
        video_path: The path of the video in the disk, whose frames are to be extracted.
        sequence_length: Number of frames to extract per video (default: 20).
        image_height: Height to resize frames to (default: 224 for Xception).
        image_width: Width to resize frames to (default: 224 for Xception).
    Returns:
        frames_list: A list containing the resized and normalized frames of the video, or None if extraction fails.
    '''
    # Declare a list to store video frames
    frames_list = []

    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file not found at {video_path}")
        return None

    # Read the video file using VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Check if the video was opened successfully
    if not video_reader.isOpened():
        print(f"Error: Could not open video file {video_path}")
        video_reader.release()
        return None

    # Get the total number of frames in the video '30' -> 30
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Ensure the video has enough frames
    if video_frames_count < sequence_length:
        print(f"Warning: Video {video_path} has only {video_frames_count} frames, less than required {sequence_length}")
        video_reader.release()
        return None

    # Calculate the interval after which frames will be sampled
    skip_frames_window = max(int(video_frames_count / sequence_length), 1)

    # Iterate to extract the specified number of frames
    # for 1 in range(20) 1...20
    for frame_counter in range(sequence_length):
        # Set the current frame position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        # Read the frame
        success, frame = video_reader.read()

        # Check if the frame was read successfully
        if not success or frame is None:
            print(f"Warning: Failed to read frame {frame_counter} from {video_path}")
            break

        # Resize the frame to the specified dimensions
        try:
            resized_frame = cv2.resize(frame, (image_width, image_height))
        except Exception as e:
            print(f"Error resizing frame {frame_counter} from {video_path}: {e}")
            break

        # Normalize the frame to [0 - 1] for model input
        normalized_frame = resized_frame / 255.0


        # Append the normalized frame to the list
        frames_list.append(normalized_frame)

    # Release the VideoCapture object
    video_reader.release()

    # Ensure the correct number of frames is extracted
    if len(frames_list) != sequence_length:
        print(f"Warning: Extracted {len(frames_list)} frames instead of {sequence_length} from {video_path}")
        return None

    # Convert to numpy array for consistency
    frames_list = np.array(frames_list)

    return frames_list

In [47]:
frames_extraction("/kaggle/input/realistic-action-recognition-ucf50/UCF50/BaseballPitch/v_BaseballPitch_g01_c01.avi")

print("Next Create Dataset...")

Next Create Dataset...


In [48]:
# does exist or not?
# /kaggle/input/realistic-action-recognition-ucf50/UCF50
os.path.exists('/kaggle/input/UCF50')

False

In [49]:
Folder = "Baseball"
File = "video 1.mp4"
Dataset = "/kaggle/input/realistic-action-recognition-ucf50/UCF50"

path = Dataset + "/" + Folder + "/" + File

print("Custom - ",path)

os.path.join(Dataset, Folder, File)

Custom -  /kaggle/input/realistic-action-recognition-ucf50/UCF50/Baseball/video 1.mp4


'/kaggle/input/realistic-action-recognition-ucf50/UCF50/Baseball/video 1.mp4'

In [50]:
# list direcotories
Dir = "/kaggle/input/realistic-action-recognition-ucf50/UCF50/JumpRope"
os.listdir(Dir)

print('')

In [51]:
class_list = ['Jump', 'Sleep']

for index, Class in enumerate(class_list):
    print(index, Class)

0 Jump
1 Sleep


In [52]:
numbers =[ number for number in range(0,50)]

print(numbers[:10])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [53]:
import os
import numpy as np

# args = /kaggle/input/realistic-action-recognition-ucf50/UCF5O/JumpRope, ['JumpRop', 'Basketball'] seq, img hw

def create_dataset(dataset_dir, classes_list, sequence_length=20, image_height=224, image_width=224, max_videos_per_class=None):
    '''
    This function extracts data for selected classes and creates the dataset.
    Args:
        dataset_dir: The directory containing the UCF-50 dataset (e.g., "/kaggle/working/UCF50_dataset/UCF50").
        classes_list: List of class names to include in the dataset.
        sequence_length: Number of frames to extract per video (default: 20).
        image_height: Height to resize frames to (default: 224 for Xception).
        image_width: Width to resize frames to (default: 224 for Xception).
        max_videos_per_class: Maximum number of videos to process per class (optional, for testing).
    Returns:
        features: A numpy array of extracted frame sequences with shape (n_videos, sequence_length, image_height, image_width, 3).
        labels: A numpy array of class indexes.
        video_files_paths: A list of video file paths.
    '''
    # /kaggle/input/realistic-action-recognition-ucf50/UCF50/BaseballPitch/v_BaseballPitch_g01_c01.avi
    # Initialize lists to store features, labels, and video file paths
    features = []
    labels = []
    video_files_paths = []

    # Check if dataset directory exists
    if not os.path.exists(dataset_dir):
        raise FileNotFoundError(f"Dataset directory not found: {dataset_dir}")

    # Iterate through all classes in the classes list
    for class_index, class_name in enumerate(classes_list):
        # dataset_dir + class name
        class_path = os.path.join(dataset_dir, class_name)
        
        # Check if class directory exists
        if not os.path.exists(class_path):
            print(f"Warning: Class directory not found: {class_path}")
            continue

        print(f'Extracting Data of Class: {class_name}')

        # Get the list of video files in the class directory
        files_list = os.listdir(class_path)

        # Limit the number of videos if specified 100
        if max_videos_per_class is not None:
            # files list = 140 -> 0-99
            files_list = files_list[:max_videos_per_class]

        # Iterate through all video files ['v1', 'v2']
        for file_name in files_list:
            # 
            video_file_path = os.path.join(class_path, file_name)

            # Extract frames using the updated frames_extraction function
            frames = frames_extraction(video_file_path, sequence_length, image_height, image_width)

            # Skip videos where frame extraction failed
            if frames is None:
                print(f"Skipping video {video_file_path} due to frame extraction failure")
                continue

            # Append the data to respective lists
            features.append(frames)
            labels.append(class_index)
            video_files_paths.append(video_file_path)

    # Convert lists to numpy arrays
    if not features:
        raise ValueError("No valid videos were processed. Check dataset or parameters.")
    
    features = np.asarray(features)
    labels = np.array(labels)

    print(f"Dataset created with {len(features)} videos")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")

    return features, labels, video_files_paths

In [55]:
information = create_dataset("/kaggle/input/realistic-action-recognition-ucf50/UCF50", ['Basketball', 'Biking'], 20,224,224)

features, labels, video_file_paths = information

print(labels)

Extracting Data of Class: Basketball
Extracting Data of Class: Biking
Dataset created with 282 videos
Features shape: (282, 20, 224, 224, 3)
Labels shape: (282,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
